In [1]:
from sqlalchemy import create_engine
engine = create_engine("sqlite:///sqlite_db_pythonsqlite.db")

In [2]:
import pandas as pd
Members = pd.read_sql_query('SELECT * FROM Members', engine)
Facilities =  pd.read_sql_query('SELECT * FROM Facilities', engine)
Bookings = pd.read_sql_query('SELECT * FROM Bookings', engine)
print(Bookings.columns)
print(Facilities.columns)
print(Members.columns)

Index(['bookid', 'facid', 'memid', 'starttime', 'slots'], dtype='object')
Index(['facid', 'name', 'membercost', 'guestcost', 'initialoutlay',
       'monthlymaintenance'],
      dtype='object')
Index(['memid', 'surname', 'firstname', 'address', 'zipcode', 'telephone',
       'recommendedby', 'joindate'],
      dtype='object')


In [3]:
#Q10: Produce a list of facilities with a total revenue less than 1000.
#The output of facility name and total revenue, sorted by revenue. Remember
#that there's a different cost for guests and members!

sql = """SELECT 
	t.name AS facility,
	SUM(total_cost) AS revenue
FROM Bookings
	RIGHT JOIN 
    (SELECT b.bookid, f.name,
     CASE WHEN memid != 0 THEN slots*membercost
	 ELSE slots*guestcost END AS total_cost
     FROM Bookings AS b
     LEFT JOIN Facilities AS f
     USING (facid)
    ) AS t
	USING (bookid)
GROUP BY facility
HAVING revenue < 1000
ORDER BY revenue;"""

Revenue = pd.read_sql_query(sql, engine)
Revenue

,facility,revenue
0,Table Tennis,180
1,Snooker Table,240
2,Pool Table,270


In [4]:
#Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order

code = """SELECT m1.surname || ', ' || m1.firstname AS member_name, 
	CASE WHEN m2.memid != 0 THEN m2.surname || ', ' || m2.firstname END AS recommender
FROM Members AS m1
	LEFT JOIN Members AS m2
	ON m1.recommendedby = m2.memid
ORDER BY member_name;"""
Recommender = pd.read_sql_query(code, engine)
Recommender

,member_name,recommender
0,"Bader, Florence","Stibbons, Ponder"
1,"Baker, Anne","Stibbons, Ponder"
2,"Baker, Timothy","Farrell, Jemima"
3,"Boothe, Tim","Rownam, Tim"
4,"Butters, Gerald","Smith, Darren"
5,"Coplin, Joan","Baker, Timothy"
6,"Crumpet, Erica","Smith, Tracy"
7,"Dare, Nancy","Joplette, Janice"
8,"Farrell, David",None
9,"Farrell, Jemima",None


In [5]:
#Q12: Find the facilities with their usage by member, but not guests

sql_code = """SELECT f.name AS facility, m.firstname || ' ' || m.surname AS fullname, SUM(slots) AS 'usage'
FROM Bookings AS b
	LEFT JOIN Facilities AS f
	USING (facid)
	LEFT JOIN Members AS m
	USING (memid)
WHERE memid != 0
GROUP BY facility, fullname;"""
member_usages = pd.read_sql_query(sql_code, engine)
member_usages

,facility,fullname,usage
0,Badminton Court,Anna Mackenzie,96
1,Badminton Court,Anne Baker,30
2,Badminton Court,Burton Tracy,6
3,Badminton Court,Charles Owen,18
4,Badminton Court,Darren Smith,432
...,...,...,...
197,Tennis Court 2,Ramnaresh Sarwin,36
198,Tennis Court 2,Tim Boothe,168
199,Tennis Court 2,Tim Rownam,18
200,Tennis Court 2,Timothy Baker,21


In [6]:
# Q13: Find the facilities usage by month, but not guests

last_code = """SELECT name AS facility, strftime('%m', starttime) AS month, SUM(slots) AS 'usage'
FROM Bookings
	LEFT JOIN Facilities
	USING (facid)
WHERE memid != 0
GROUP BY facility, month;"""
member_monthly_usage = pd.read_sql_query(last_code, engine)
member_monthly_usage

,facility,month,usage
0,Badminton Court,07,165
1,Badminton Court,08,414
2,Badminton Court,09,507
3,Massage Room 1,07,166
4,Massage Room 1,08,316
5,Massage Room 1,09,402
6,Massage Room 2,07,8
7,Massage Room 2,08,18
8,Massage Room 2,09,28
9,Pool Table,07,110
